In [ ]:
!pip install qiskit qiskit-ibm-provider numpy scipy matplotlib

# SYK–VQE Pipeline (expanded)

This notebook includes the VQE + MERA + QSE pipeline and adds SWAP-test and Classical Shadows cells for shot-based Renyi-2 estimation on IBM backends.

Run cells sequentially.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from math import log
from qiskit import Aer, transpile, QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace, SparsePauliOp
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA, SLSQP
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.circuit import ParameterVector

algorithm_globals.random_seed = 42
np.random.seed(42)


## SYK toy Hamiltonian

In [ ]:
pauli_list = [
    ( -1.2, "ZZ"),
    (  0.6, "XI"),
    (  0.4, "IZ"),
    (  0.2, "YI")
]
H = SparsePauliOp.from_list([(p, c) for c, p in pauli_list])
print('Hamiltonian:', H)
dense_H = H.to_matrix()
eigvals, eigvecs = np.linalg.eigh(dense_H)
print('Exact ground energy:', eigvals[0])


## MERA-inspired ansatz (true-MERA style)

In [ ]:
def build_true_mera(n_qubits, depth=2):
    theta = ParameterVector('mera', depth*3*n_qubits)
    qc = QuantumCircuit(n_qubits)
    p = 0
    for d in range(depth):
        for q in range(0, n_qubits-1, 2):
            qc.crx(theta[p], q, q+1); p+=1
            qc.cry(theta[p], q, q+1); p+=1
            qc.crz(theta[p], q, q+1); p+=1
        for q in range(1, n_qubits, 2):
            qc.cx(q, q-1)
            qc.ry(theta[p], q-1); p+=1
            qc.barrier()
    return qc

ansatz = build_true_mera(H.num_qubits, depth=2)
try:
    display(ansatz.draw('mpl'))
except:
    print(ansatz)


## VQE run (statevector simulator)

In [ ]:
backend = Aer.get_backend('aer_simulator_statevector')
qi = QuantumInstance(backend, seed_simulator=42, seed_transpiler=42)
optimizer = SLSQP(maxiter=200)
energy_history = []
def callback(eval_count, params, energy, std):
    energy_history.append(float(energy))

vqe = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=qi, callback=callback)
result = vqe.compute_minimum_eigenvalue(H)
print('VQE energy:', result.eigenvalue.real)
print('Exact ground:', eigvals[0])
plt.plot(energy_history, 'o-'); plt.axhline(eigvals[0], color='r', linestyle='--'); plt.title('VQE convergence'); plt.show()


## Quantum Subspace Expansion (QSE)

In [ ]:
def run_qse(H_op, ansatz, params):
    sv = Statevector.from_instruction(ansatz.bind_parameters(params))
    basis = [sv]
    for lbl, _ in H_op.to_list()[:min(6,len(H_op.to_list()))]:
        mat = SparsePauliOp.from_list([(lbl,1.0)]).to_matrix()
        vec = mat @ sv.data
        basis.append(Statevector(vec/np.linalg.norm(vec)))
    S = np.array([[np.vdot(bi.data, bj.data) for bj in basis] for bi in basis])
    Hs = np.array([[np.vdot(bi.data, H_op.to_matrix() @ bj.data) for bj in basis] for bi in basis])
    vals, _ = np.linalg.eigh(np.linalg.inv(S) @ Hs)
    return np.sort(vals)

qse_vals = run_qse(H, ansatz, result.optimal_point)
print('QSE refined eigenvalues (sample):', qse_vals[:4])


## Entanglement (von Neumann and Renyi-2)

In [ ]:
sv = Statevector.from_instruction(ansatz.bind_parameters(result.optimal_point))
rho = sv.to_density_matrix()
rhoA = partial_trace(rho, [1])
eigvals_rhoA = np.real_if_close(np.linalg.eigvals(rhoA.data))
eigvals_rhoA = np.clip(eigvals_rhoA,0,1)
S_vN = -np.sum([p*np.log(p) for p in eigvals_rhoA if p>1e-12])
purity = np.sum(eigvals_rhoA**2)
S2 = -np.log(purity)
print('von Neumann S:', S_vN)
print('Renyi-2 S2:', S2)


## SWAP-test circuit (shot-based) to estimate Renyi-2 (purity)

This cell constructs the SWAP-test between two copies of the ansatz state for subsystem A and runs it on a qasm backend (simulator or IBM device).

In [ ]:
# Controlled-swap decomposition helper
def controlled_swap_decomp(qc, control, a, b):
    qc.cx(b, a)
    qc.ccx(control, a, b)
    qc.cx(b, a)

def build_swap_test_circuit(ansatz_bound, subsystem_A):
    n = ansatz_bound.num_qubits
    total_q = 2*n + 1
    qc = QuantumCircuit(total_q, 1)
    qc.compose(ansatz_bound, qubits=list(range(0, n)), inplace=True)
    qc.compose(ansatz_bound, qubits=list(range(n, 2*n)), inplace=True)
    anc = 2*n
    qc.h(anc)
    for idx in subsystem_A:
        controlled_swap_decomp(qc, anc, idx, n + idx)
    qc.h(anc)
    qc.measure(anc, 0)
    return qc

# bind ansatz with optimal params
circ_bound = ansatz.bind_parameters([result.optimal_point[p] for p in ansatz.parameters])
subsystem_A = [0]  # measure purity of qubit 0
swap_qc = build_swap_test_circuit(circ_bound, subsystem_A)
print('SWAP-test circuit depth:', swap_qc.depth())
# run on qasm simulator for demonstration
backend_qasm = Aer.get_backend('aer_simulator')
job = backend_qasm.run(transpile(swap_qc, backend_qasm), shots=2048)
counts = job.result().get_counts()
P0 = counts.get('0', 0) / 2048
purity_swap = 2.0 * P0 - 1.0
print('Swap-test purity estimate (sim):', purity_swap)
print('Exact purity (statevector):', purity)


## Classical shadows estimator (shot-based) to reconstruct rho and estimate purity

In [ ]:
import random
def apply_basis_rotation_for_label(qc, qubit, basis):
    if basis == 'X':
        qc.h(qubit)
    elif basis == 'Y':
        qc.sdg(qubit); qc.h(qubit)

def projector_for_basis_outcome(basis, outcome):
    if basis == 'Z':
        return np.array([[1,0],[0,0]]) if outcome == 0 else np.array([[0,0],[0,1]])
    if basis == 'X':
        return 0.5 * (np.array([[1,0],[0,1]]) + (np.array([[0,1],[1,0]]) if outcome==0 else -np.array([[0,1],[1,0]])))
    if basis == 'Y':
        return 0.5 * (np.array([[1,0],[0,1]]) + (np.array([[0,-1j],[1j,0]]) if outcome==0 else -np.array([[0,-1j],[1j,0]])))

def classical_shadows_estimator(ansatz_bound, snapshot_rounds=200):
    n = ansatz_bound.num_qubits
    rho_tilde = np.zeros((2**n, 2**n), dtype=complex)
    backend = Aer.get_backend('aer_simulator')
    for s in range(snapshot_rounds):
        bases = [random.choice(['X','Y','Z']) for _ in range(n)]
        qc = QuantumCircuit(n, n)
        qc.compose(ansatz_bound, qubits=list(range(n)), inplace=True)
        for q_idx,b in enumerate(bases):
            apply_basis_rotation_for_label(qc, q_idx, b)
        qc.measure(range(n), range(n))
        job = backend.run(transpile(qc, backend), shots=1)
        counts = job.result().get_counts()
        bstr = list(counts.keys())[0]
        bits = list(map(int, list(reversed(bstr))))
        single_snapshot = np.array([1.0], dtype=complex)
        for q_idx in range(n):
            P = projector_for_basis_outcome(bases[q_idx], bits[q_idx])
            Omega = (3.0 * P - np.eye(2)) / 2.0
            single_snapshot = np.kron(single_snapshot, Omega)
        rho_tilde += single_snapshot
    rho_tilde = rho_tilde / float(snapshot_rounds)
    rho_tilde = 0.5 * (rho_tilde + rho_tilde.conj().T)
    eigs = np.real_if_close(np.linalg.eigvals(rho_tilde))
    eigs = np.clip(eigs, 0, 1)
    if eigs.sum()>0:
        eigs = eigs / eigs.sum()
    purity_est = float(np.trace(rho_tilde @ rho_tilde))
    nonzero = eigs[eigs > 1e-12]
    S_vN = -float(np.sum(nonzero * np.log(nonzero))) if len(nonzero)>0 else 0.0
    return rho_tilde, purity_est, S_vN

rho_tilde, purity_shadow, S_vN_shadow = classical_shadows_estimator(circ_bound, snapshot_rounds=200)
print('Classical shadows purity estimate:', purity_shadow, 'S_vN estimate:', S_vN_shadow)


## Level spacing statistics (histogram vs GUE/Poisson)

In [ ]:
def plot_level_statistics(vals):
    spacings = np.diff(np.sort(vals))
    spacings /= np.mean(spacings)
    plt.hist(spacings, bins=15, density=True, alpha=0.6)
    x = np.linspace(0,3,200)
    plt.plot(x, (np.pi/2)*x*np.exp(-np.pi*x**2/4), 'r-', label='GUE')
    plt.plot(x, np.exp(-x), 'k--', label='Poisson')
    plt.legend(); plt.show()

plot_level_statistics(eigvals)


## Running on IBM Quantum

To run SWAP-test or shadow circuits on IBM devices:

```python
from qiskit_ibm_provider import IBMProvider
IBMProvider.save_account('MY_API_TOKEN')
provider = IBMProvider()
backend = provider.get_backend('ibmq_qasm_simulator')
job = backend.run(transpile(swap_qc, backend), shots=2048)
print(job.result().get_counts())
```
Ensure your account token is kept private.